In [1073]:
import numpy as np
from math import sqrt

# Helper functions
def sigmoid(x):
    x = np.clip(x, -500, 500) # not to run into overflow error
    return 1 / (1 + np.exp(-x))

def sigmoid_derived(x):
    x = np.clip(x, -100, 100) # not to run into overflow error
    return np.exp(x)/(1 + np.exp(x))**2

class MyNeuralNetwork():
    '''
    Activation function in hidden layers is sigmoid. 
    Output layer is linear for now (regression problems only).
    '''
    
    def __init__(self, architecture):

        print("Initializing network with architecture:", architecture)
        
        self.architecture = architecture  # Array describing network structure, first is input, last is output all others are hidden
        self.input_size = architecture[0] # Input features size
        self.layers = len(architecture)   # Number of Layers
        
        self.W = [] # list of weight matrices
        self.initialize_W()
        
    def initialize_W(self):
        '''
        Initializes weights from N(0, 1/sqrt(input_features_size))
        '''
        for i in range(self.layers - 1):
            tmp_W = np.random.normal(loc = 0, scale = 1/sqrt(self.input_size), size = self.architecture[i:i+2])
            self.W.append(tmp_W)
    
    def feedforward_one_step(self, x, layer):
        y = np.dot(x, self.W[layer])
        if layer == self.layers - 2:  # Do not apply activation function in output layer calculations
            return y
        else:                         # Apply activation function
            return sigmoid(y)
    
    def feedforward(self, x):
        '''For a given input vector column calculate output'''
        for layer in range(self.layers - 1):
            x = self.feedforward_one_step(x, layer)
        return x
    
    def feedforward_full(self, x):
        '''For a given input vector column feature vector for each layer'''
        list_x = []
        x.shape = (1, x.size)
        list_x.append(x)
        
        for layer in range(self.layers - 1):
            x = self.feedforward_one_step(x, layer)
            x.shape = (1, x.size)
            list_x.append(x)

        return list_x
    
    def backpropagation(self, x, y, alpha = 0.01):
        list_x = self.feedforward_full(x) # Ima indekse 0, 1, 2, 3
        gradients = []
        delta = 1
        for layer in range(self.layers - 2, -1, -1):  ## ide 2, 1, 0
            new_gradient = np.dot(delta, list_x[layer])
            new_gradient = np.transpose(new_gradient)
            gradients.append(new_gradient)
            
            if(layer > 0):
                psi = np.dot(list_x[layer-1], self.W[layer-1])
                sigm_d = np.eye(psi.size) * sigmoid_derived(psi)
                delta = np.dot(np.dot(sigm_d, self.W[layer]), delta)
        gradients.reverse()
        
        error = list_x[-1] - y
        # We have gradients, now we just apply them
        for i, W in enumerate(self.W):
            '''
            print("Current Weights: \n", self.W[i], 
                  "\nNet Estimate", list_x[-1],
                  "\nError: \n", error,
                 "\nGradient: \n", gradients[i],
                 "\n Update: \n", - alpha * error * gradients[i])
            '''
            self.W[i] = self.W[i] - alpha * error * gradients[i]
        return

Lets learn function f(x, y) = x^2 + y

In [1130]:
def f(x, y):
    return x**2 + y

In [1162]:
x_list = []
y_list = []

for i in range(50):
    for j in range(50):
        x_list.append(np.array([(i+1), (j+1)]))
        y_list.append(f(i+1, j+1))
        


In [1163]:
from statistics import stdev
st = stdev(list(range(1,51)))

In [1176]:
net = MyNeuralNetwork([2, 10, 5, 1])

print("f(2, 3): ", f(2, 3))
print("net(2, 3): ", net.feedforward(np.array([2, 3])))

print("f(10, 5): ", f(10, 5))
print("net(10, 5): ", net.feedforward(np.array([10, 5])))

Initializing network with architecture: [2, 10, 5, 1]
f(2, 3):  7
net(2, 3):  [-1.19459527]
f(10, 5):  105
net(10, 5):  [-1.33171988]


In [1177]:
for x, y in zip(x_list, y_list):
    net.backpropagation(x, y, alpha = 0.001)

In [1178]:

print("f(2, 3): ", f(2, 3))
print("net(2, 3): ", net.feedforward(np.array([2/st, 3/st])))

print("f(10, 5): ", f(10, 5))
print("net(10, 5): ", net.feedforward(np.array([10/st, 5/st])))

f(2, 3):  7
net(2, 3):  [2202.0057578]
f(10, 5):  105
net(10, 5):  [2202.85841901]


[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 26,
 27,
 28

In [848]:
net.W

[array([[-0.04074726],
        [ 0.59936029]])]

In [979]:
net.backpropagation(np.array([0, 1]), 1, alpha = 0.1)
net.backpropagation(np.array([0, 2]), 2, alpha = 0.1)
net.backpropagation(np.array([0, 3]), 3, alpha = 0.1)
net.backpropagation(np.array([0, 4]), 4, alpha = 0.1)
net.backpropagation(np.array([0, 5]), 5, alpha = 0.1)
net.backpropagation(np.array([2, 0]), 2, alpha = 0.1)
net.backpropagation(np.array([2, 1]), 3, alpha = 0.1)
net.backpropagation(np.array([2, 2]), 4, alpha = 0.1)
net.backpropagation(np.array([2, 3]), 5, alpha = 0.1)
net.backpropagation(np.array([2, 4]), 6, alpha = 0.1)

net.W

[array([[0.99999979],
        [1.00000008]])]

In [982]:
y_list

[0.013943538237355403,
 1.005371345353725,
 2.0482055775049743,
 3.007667901322303,
 4.050773092017647,
 5.0783817312817074,
 6.028328563453111,
 7.036165847051284,
 8.073986436858306,
 9.03144170500435,
 10.003992203166831,
 11.080844019316839,
 12.064827341074048,
 13.000824172774074,
 14.073482414508312,
 15.07602807125212,
 16.07246109064991,
 17.099145345879677,
 18.043491822871886,
 19.099975439376774,
 20.02579695063484,
 21.004203401767462,
 22.04364472673093,
 23.074020302927213,
 24.026314562717722,
 25.050070442591505,
 26.045736415378965,
 27.089408716224174,
 28.02419496895009,
 29.091892710085283,
 30.090438083776874,
 31.05685087745167,
 32.05938560773866,
 33.06199965223573,
 34.00198584878968,
 35.0115642438296,
 36.01937098073409,
 37.01694673227295,
 38.022408361045706,
 39.04358235064475,
 40.08625504638864,
 41.06363219718082,
 42.05057251713565,
 43.009506524605534,
 44.08469963390282,
 45.003231774868816,
 46.061388121312795,
 47.01991575586227,
 48.0969234898270